In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np 
import pandas as pd
from keras.applications import xception

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
weather_path = "/content/drive/MyDrive/dataset/"
classes = os.listdir(weather_path)
classes

['sunrise', 'shine', 'cloudy', 'foggy', 'rainy']

In [ ]:
image_size = (224, 224)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    weather_path,
    validation_split=0.2,          
    subset="training",             
    seed=1337,                     
    image_size=image_size,         
    batch_size=batch_size,         
    label_mode="categorical",      
    class_names=classes
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    weather_path,
    validation_split=0.2,
    subset="validation",          
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode="categorical",
    class_names=classes
)


Found 1500 files belonging to 5 classes.
Using 1200 files for training.
Found 1500 files belonging to 5 classes.
Using 300 files for validation.


In [ ]:
input_shape = [224, 224]


model = tf.keras.applications.Xception(
    input_shape=input_shape+[3],weights='imagenet',include_top=False
)

model.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

In [ ]:
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

x=Flatten()(model.output)
predictions=Dense(5,activation='softmax')(x)

model1=Model(inputs=model.input,outputs=predictions)

In [ ]:
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           

In [ ]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model1.fit(train_ds, epochs=3, verbose=1, validation_data=val_ds)

Epoch 1/3
38/38 [==============================] - 60s 1s/step - loss: 0.9990 - accuracy: 0.6975 - val_loss: 22.5580 - val_accuracy: 0.2867
Epoch 2/3
38/38 [==============================] - 53s 1s/step - loss: 0.2822 - accuracy: 0.9233 - val_loss: 24.3762 - val_accuracy: 0.5867
Epoch 3/3
38/38 [==============================] - 53s 1s/step - loss: 0.2495 - accuracy: 0.9367 - val_loss: 3.9667 - val_accuracy: 0.6967


In [ ]:
 model1.save('Xception.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('Xception.h5')

In [ ]:
from tensorflow.keras.preprocessing import image

img = image.load_img('/content/drive/MyDrive/test_img/test1.jpg', target_size= (224, 224))

In [ ]:
img = image.img_to_array(img)

In [ ]:
img = np.expand_dims(img, axis = 0)

In [ ]:
result = model.predict(img)

In [ ]:
classes[np.argmax(result)]

'rainy'